<a href="https://colab.research.google.com/github/yoojinleee/CarGame/blob/main/CarGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pygame #A code to install the pygame for users who may not have it installed. 

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pygame
import random
import sys
import time

# Initializing the game
pygame.init()

# Making the settings for the screen display
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Car Racing Game")

# Frame rate for moving car
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 48)

#Colors
WHITE = (255, 255, 255)
Black = (0,0,0)
Red = (255, 0, 0)
Blue = (0, 0, 255)
Yellow = (255, 255, 0)
road_line_offset = 0 
line_height = 40 
line_spacing = 60 

#Car
car_width, car_height = 50, 100
car = pygame.Rect(WIDTH//2, HEIGHT - 120, car_width, car_height)

# Making the car move
car_speed = 5

#Obstacle
obstacle_width, obstacle_height = 50, 100
obstacles = []
obstacle_speed = 5

#Game Activity
running = True
game_active = True
game_state = "playing"
last_math_time = time.time ()
math_interval = 30

score = 0 

num1, num2 = 0,0
answer = 0
user_input = ""

def draw_background():
  global road_line_offset
#setting the colors 
  road_color = (50, 50, 50)
  grass_color = (34, 139, 34)
  line_color = Yellow
#creating the road boundaries
  road_width = 400 
  road_left = (WIDTH - road_width) // 2
  road_right = road_left + road_width 
#drawing the grass as the borders 
  pygame.draw.rect(screen, grass_color, (0, 0, road_left, HEIGHT))
  pygame.draw.rect(screen, grass_color, (road_right, 0, WIDTH - road_right, HEIGHT))

#drawing the road
  pygame.draw.rect(screen, road_color, (road_left, 0, road_width, HEIGHT))

#detailing the road with yellow lines running down the middle 
  num_lines = HEIGHT // (line_height + line_spacing) + 2
  for i in range(num_lines): 
    y = (i * (line_height + line_spacing)) + road_line_offset
    pygame.draw.rect(screen, line_color, (WIDTH//2 - 5, y % HEIGHT, 10, line_height))

   #scrolling effect for the road 
  road_line_offset += obstacle_speed
  road_line_offset %= (line_height + line_spacing) 

def draw_car():
  #drawing the car body
  pygame.draw.rect(screen, Blue, (car.x, car.y + 20, car.width, car.height -20))
# drawing the top part of the car 
  pygame.draw.rect(screen, (173, 216, 230), (car.x + 10, car.y, car.width - 20, 30))

  #drawing the wheels 
  wheel_radius = 10 
  pygame.draw.circle(screen, Black, (car.x + 10, car.y + car.height), wheel_radius) 
  pygame.draw.circle(screen, Black, (car.x + car.width - 10, car.y + car.height), wheel_radius)
  pygame.draw.circle(screen, Black, (car.x + 10, car.y + car.height - 30), wheel_radius)
  pygame.draw.circle(screen, Black, (car.x + car.width - 10, car.y + car.height - 30), wheel_radius)

def draw_obstacles():
  for obs in obstacles:
    pygame.draw.rect(screen, Red, obs)

def spawn_obstacle():
  road_left = (WIDTH - 400) // 2
  road_right = (WIDTH + 400) // 2 - obstacle_width 
  x = random.randint(road_left, road_right)
  new_obstacle = pygame.Rect(x, -obstacle_height, obstacle_width, obstacle_height)
  obstacles.append(new_obstacle)

def move_obstacles():
 global score
 for obs in obstacles: 
   obs.y += obstacle_speed
   for obs in obstacles[:]:
     if obs.y >= HEIGHT: 
       score += 1 
       obstacles.remove(obs)

def check_collision():
  for obs in obstacles:
    if car.colliderect(obs):
      return True
  return False

def show_message(message, delay=2000):
  screen.fill(WHITE)
  text = font.render(message, True, Black)
  rect = text.get_rect(center=(WIDTH//2, HEIGHT//2))
  screen.blit(text, rect)
  pygame.display.flip()
  pygame.time.delay(delay)

def start_math_quiz():
  global num1, num2, answer, user_input, game_state
  num1 = random.randint(1,10)
  num2 = random.randint(1,10)
  answer = num1 * num2
  user_input = ""
  game_state = "math_quiz"

while running:
  clock.tick(60)
  draw_background()

  for event in pygame.event.get():
    if event.type == pygame.QUIT:
      running = False

    if game_state == "math_quiz":
      if event.type == pygame.KEYDOWN: 
        if event.key == pygame.K_BACKSPACE:
          user_input = user_input[:-1]
        elif event.key == pygame.K_RETURN:
          if user_input.isdigit():
            if int(user_input) == answer:
              score += 5
              game_state = "playing"
              last_math_time = time.time()
            else:
              show_message("Wrong! You died.")
              game_active = False
              game_state = "game_over"
        else:
          user_input += event.unicode

    elif game_state == "game_over":
      if event.type == pygame.KEYDOWN:
        if event.key == pygame.K_r:
          car.x = WIDTH//2
          car.y = HEIGHT - 120
          obstacles.clear()
          last_math_time = time.time()
          game_active = True
          game_state = "playing"
          score = 0
        elif event.key == pygame.K_q:
          pygame.quit()
          sys.exit()


  if game_state == "math_quiz":
    question = font.render(f"What is {num1} x {num2}?", True, Black)
    input_box = font.render(user_input, True, Red)
    screen.blit(question, (WIDTH//2 - question.get_width()//2, HEIGHT//2-50))
    screen.blit(input_box, (WIDTH//2 - input_box.get_width()//2, HEIGHT//2+10))

  elif game_state == "playing" and game_active:
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT] and car.left > (WIDTH - 400) // 2:
      car.x -= car_speed
    if keys[pygame.K_RIGHT] and car.right < (WIDTH + 400) // 2:
      car.x += car_speed

    if random.randint(1,20)==1:
      spawn_obstacle()

    move_obstacles()
    draw_car()
    draw_obstacles()
    score_text = font.render(f"Score: {score}", True, Black)
    screen.blit(score_text, (10,10))

    if check_collision():
      show_message("You crashed!")
      game_active = False
      game_state = "game_over"

    #Time-based math challenge
    if time.time() - last_math_time > math_interval:
      start_math_quiz()
    
    pygame.display.flip()
    

  elif game_state == "game_over":
    screen.fill(WHITE)
    msg = font.render("Game Over! Press R to restart or Q to quit", True, Black)
    screen.blit(msg, (WIDTH//2 - msg.get_width()//2, HEIGHT//2))
    
  pygame.display.flip()

pygame.quit()
sys.exit()

SystemExit: 

/Users/faithpark/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
